<a href="https://colab.research.google.com/github/caltunay/tr_lyric_generator/blob/main/Eksi_Gundem_Scrape_%26_Assign_Genre_%26_Generate_for_each_Seed_%26_Publish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from datetime import datetime
import urllib 
import tweepy
import webbrowser
import time

In [ ]:
api_key = ''
api_secret_key = ''

callback_uri = 'oob'

auth = tweepy.OAuthHandler(api_key, api_secret_key, callback_uri)
redirect_url = auth.get_authorization_url()
print(redirect_url)

In [ ]:
user_pin = input('Authorization PIN: ')

Authorization PIN: 3108021


In [ ]:
auth.get_access_token(user_pin)

In [ ]:
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [ ]:
# model vocabs
rock_vocab = ['\n', ' ', '!', "'", ',', '.', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', 'z', 'â', 'ç', 'î', 'ö', 'û', 'ü', 'ğ', 'ı', 'ş', '̇', '‘', '’', '…']
tsm_vocab = ['\n', ' ', '!', "'", ',', '.', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', 'z', 'â', 'ç', 'î', 'ö', 'û', 'ü', 'ğ', 'ı', 'ş', '̇', '‘', '’', '…']
rap_vocab = ['\n', ' ', '!', ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'y', 'z', 'ç', 'ö', 'ü', 'ğ', 'ı', 'ş', '̇', '‘', '’', '…']
arabesk_vocab = ['\n', ' ', '!', "'", ',', '.', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', 'z', 'â', 'ç', 'é', 'î', 'ö', 'û', 'ü', 'ğ', 'ı', 'ş', '̇', '–', '‘', '’', '…']
pop_vocab = ['\n', ' ', '!', "'", ',', '.', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'y', 'z', 'â', 'ç', 'ö', 'û', 'ü', 'ğ', 'ı', 'ş', '̇', '‘', '’', '…']

In [ ]:
# get most popular topics from eksisozluk

rock_twit_eligible_gundem = list()
tsm_twit_eligible_gundem = list()
rap_twit_eligible_gundem = list()
arabesk_twit_eligible_gundem = list()
pop_twit_eligible_gundem = list()


f = urllib.request.urlopen(f'https://eksisozluk.com/basliklar/gundem?p=1')
soup = bs(f, 'html.parser')
topics = soup.find('ul', class_='topic-list').find_all('li')
months = 'aralık ocak şubat mart nisan mayıs haziran temmuz ağustos eylül ekim kasım'.split()

for topic in topics:
    
    try:
        gundem_madde = topic.find('a').get_text()[:topic.find('a').get_text().rfind(' ')].lower()
        str_len = len(gundem_madde)

        if all(word not in months for word in gundem_madde.split()):

            # add to eligible genre lists 
            if str_len == 29:
                if all(char in rock_vocab for char in gundem_madde):
                    rock_twit_eligible_gundem.append(gundem_madde + ' ')

                if all(char in tsm_vocab for char in gundem_madde):
                    tsm_twit_eligible_gundem.append(gundem_madde + ' ')

                if all(char in rap_vocab for char in gundem_madde):
                    rap_twit_eligible_gundem.append(gundem_madde + ' ')

                if all(char in arabesk_vocab for char in gundem_madde):
                    arabesk_twit_eligible_gundem.append(gundem_madde + ' ')

                if all(char in pop_vocab for char in gundem_madde):
                    pop_twit_eligible_gundem.append(gundem_madde + ' ')


            elif str_len >= 30:
                if all(char in rock_vocab for char in gundem_madde):
                    rock_twit_eligible_gundem.append(gundem_madde[:30])

                if all(char in tsm_vocab for char in gundem_madde):
                    tsm_twit_eligible_gundem.append(gundem_madde[:30])

                if all(char in rap_vocab for char in gundem_madde):
                    rap_twit_eligible_gundem.append(gundem_madde[:30])

                if all(char in arabesk_vocab for char in gundem_madde):
                    arabesk_twit_eligible_gundem.append(gundem_madde[:30])

                if all(char in pop_vocab for char in gundem_madde):
                    pop_twit_eligible_gundem.append(gundem_madde[:30])
    except:
        pass

In [ ]:
tweets = [rock_twit_eligible_gundem,tsm_twit_eligible_gundem,rap_twit_eligible_gundem,arabesk_twit_eligible_gundem,pop_twit_eligible_gundem]
genres = 'rock tsm rap arabesk pop'.split()
cols = ['baslik','genre']
df = pd.DataFrame(columns = cols)

for tweet, genre in zip(tweets, genres):
    for seed in tweet:
        temp_list = [[seed, genre]]
        temp_df = pd.DataFrame(temp_list, columns = cols)

        df = pd.concat([df, temp_df])

In [ ]:
def seed_partition(df):
    
    tsm_seeds = list()
    rap_seeds = list()
    rock_seeds = list()
    pop_seeds = list()
    arabesk_seeds = list()
    unique_seeds_sorted_df = pd.DataFrame(df.groupby(['baslik']).size()).reset_index().sort_values(by = [0, 'baslik'])

    for i in range(len(unique_seeds_sorted_df)):

        temp_seed = unique_seeds_sorted_df['baslik'].iloc[i]    
        seed_possible_genres = list(df[df['baslik'] == temp_seed]['genre'])

        genre_length = dict()
        for genre in seed_possible_genres:
            if genre == 'rock':
                genre_length['rock'] = len(rock_seeds)
            elif genre == 'pop':
                genre_length['pop'] = len(pop_seeds)
            elif genre == 'arabesk':
                genre_length['arabesk'] = len(arabesk_seeds)
            elif genre == 'rap':
                genre_length['rap'] = len(rap_seeds)
            elif genre == 'tsm':
                genre_length['tsm'] = len(tsm_seeds)

        seed_genre_sorted_df = pd.melt(pd.DataFrame(genre_length, index = [1])).sort_values(by = ['value'], ascending = True)
        seed_genre_choice = np.random.choice(seed_genre_sorted_df[seed_genre_sorted_df['value']==seed_genre_sorted_df.iloc[0,1]]['variable'])

        if seed_genre_choice == 'tsm':
            tsm_seeds.append(temp_seed)
        elif seed_genre_choice == 'rap':
            rap_seeds.append(temp_seed)
        elif seed_genre_choice == 'rock':
            rock_seeds.append(temp_seed)
        elif seed_genre_choice == 'arabesk':
            arabesk_seeds.append(temp_seed)
        elif seed_genre_choice == 'pop':
            pop_seeds.append(temp_seed)

    seeds_dict =  {'tsm':tsm_seeds,
                   'rap':rap_seeds,
                   'rock':rock_seeds,
                   'arabesk':arabesk_seeds,
                   'pop':pop_seeds}
    
    return seeds_dict

In [ ]:
seeds = seed_partition(df)

In [ ]:
seeds.keys()

dict_keys(['tsm', 'rap', 'rock', 'arabesk', 'pop'])

In [ ]:
seeds

{'arabesk': ["berat albayrak'ın öldürülmüş o",
  'genetik diyor ki türkiye türk ',
  'insanın kendine itiraf edemedi',
  'sigara içmeyen insanların sıkı',
  'yarın sabah erken kalkıp çalış'],
 'pop': ["ciddi ciddi erdoğan'ı destekle",
  'boşa gittiğine en çok üzülünen',
  'hollanda gençlerinin tükenmişl',
  'mama verince yemeyen sokak ked',
  'yalnız başına yapılmış en güze'],
 'rap': ['2. el otomobil piyasasında sat',
  '792 liseden boğaziçi bildirisi',
  'devlet isterse 6 ayda nükleer ',
  'çiftçinin cebinde iphone 6 tel',
  'yurt dışında yaşamanın en güze'],
 'rock': ['bir giysiyi beş yıldan fazla g',
  'chp üyeliğinden istifa ediyoru',
  'kız arkadaşına yemek yapan erk',
  'omurilik soğanını bilmeyen dok',
  'özgür irademiz var mı sorunsal'],
 'tsm': ["akp'nin ülkeyi getirdiği nokta",
  'bir kadının sahip olması gerek',
  'güzellik hariç erkekleri aşık ',
  'sigara içen kişilerle sevgili ',
  'uzaylılar yabancı uyruklu mu s']}

In [ ]:
from tensorflow.keras.models import load_model
import sys
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
tsm_model = load_model('/content/drive/MyDrive/Colab Notebooks/NLP/Song Lyric Generator/RESUMED_TRAINING MODEL 28-Jan-2021 (06:42:38) TR_TSM_lyric_generator(seed len 30).hdf5')
rap_model = load_model('/content/drive/MyDrive/Colab Notebooks/NLP/Song Lyric Generator/MODEL 02-Feb-2021 (04:53:08) TR_RAP_lyric_generator(seed len 30).hdf5')
rock_model = load_model('/content/drive/MyDrive/Colab Notebooks/NLP/Song Lyric Generator/MODEL 01-Feb-2021 (04:29:31) TR_ROCK_lyric_generator(seed len 30).hdf5')
pop_model = load_model('/content/drive/MyDrive/Colab Notebooks/NLP/Song Lyric Generator/MODEL 01-Feb-2021 (04:51:55) TR_POP_lyric_generator(seed len 30).hdf5')
arabesk_model = load_model('/content/drive/MyDrive/Colab Notebooks/NLP/Song Lyric Generator/MODEL 01-Feb-2021 (02:25:37) TR_ARABESK_lyric_generator(seed len 30).hdf5')

In [ ]:
# vocab dicts
tsm_char_to_ind = {char:ind for ind, char in enumerate(tsm_vocab)}
tsm_ind_to_char = {ind:char for ind, char in enumerate(tsm_vocab)}

rock_char_to_ind = {char:ind for ind, char in enumerate(rock_vocab)}
rock_ind_to_char = {ind:char for ind, char in enumerate(rock_vocab)}

pop_char_to_ind = {char:ind for ind, char in enumerate(pop_vocab)}
pop_ind_to_char = {ind:char for ind, char in enumerate(pop_vocab)}

arabesk_char_to_ind = {char:ind for ind, char in enumerate(arabesk_vocab)}
arabesk_ind_to_char = {ind:char for ind, char in enumerate(arabesk_vocab)}

rap_char_to_ind = {char:ind for ind, char in enumerate(rap_vocab)}
rap_ind_to_char = {ind:char for ind, char in enumerate(rap_vocab)}

In [ ]:
def sample(preds, temperature = 1.0):
    # helper to sample index from a prob array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature

    preds = np.exp(preds) / np.sum(np.exp(preds))
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_w_seed(all_seeds):
    max_len = 30
    all_generated_lyrics = list()
    lyrics_dict = dict()

    for genre in all_seeds:
        print('\n\n==============================')
        print(f'============={genre}=============')
        print('==============================')
        
        if genre == 'tsm':
            model = tsm_model
            vocab = tsm_vocab
            ind_to_char = tsm_ind_to_char
            char_to_ind = tsm_char_to_ind
        elif genre == 'arabesk':
            model = arabesk_model
            vocab = arabesk_vocab
            ind_to_char = arabesk_ind_to_char
            char_to_ind = arabesk_char_to_ind
        elif genre == 'pop':
            model = pop_model
            vocab = pop_vocab
            ind_to_char = pop_ind_to_char
            char_to_ind = pop_char_to_ind
        elif genre == 'rap':
            model = rap_model
            vocab = rap_vocab
            ind_to_char = rap_ind_to_char
            char_to_ind = rap_char_to_ind        
        elif genre == 'rock':
            model = rock_model
            vocab = rock_vocab
            ind_to_char = rock_ind_to_char
            char_to_ind = rock_char_to_ind
        
        genre_lyrics = list()

        for sentence in seeds[genre]:
            print('\n\n=======SEED========\n')
            sentence = sentence[0:max_len]
            temperature = np.random.uniform(0.35, 0.85)
            print(f'temperature: {temperature}\n\n')
            generated = ''
            generated += sentence
            
            sys.stdout.write(generated)

            for i in range(250):
                x_pred = np.zeros((1, max_len, len(vocab)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_to_ind[char]] = 1

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = ind_to_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()

            genre_lyrics.append(generated)

        lyrics_dict[genre] = genre_lyrics
    print()
    return lyrics_dict

In [ ]:
lyrics_d = generate_w_seed(seeds)



=============tsm=============


=======SEED========

temperature: 0.5021910102416174


akp'nin ülkeyi getirdiği nokta

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


n
bir çare ararsan da benim allah’an buluşuruz
biz ayrilamayiz
eller ayirsa bile
yollar ayırsa bile
yıllar ayirsa bile
biz ayrilamayiz
eller ayirsa bile
yollar ayirsa bile
biz ayrılamayız
ellerini taşına dilde
sevginin barajında
bir gün görmedim gönü

=======SEED========

temperature: 0.653049897619735


bir kadının sahip olması gerek
her yerde senin ismini
ben inanmam hakım söyle
sen nerdesin ben nerdeyim
hani nerde sevdiğim kadeh birbirimize
kul olur üzme beni
her gece bir tutkunun
sesin demedin de gözlerin yaşı
saraba sil gül görmesin diye
gözüm aldattım sesinden hiç çıktı gön

=======SEED========

temperature: 0.6996802440466048


güzellik hariç erkekleri aşık olmadı
ömrüm hiç gibi geçti
derdin ne diye soran olmadı
gel de böyle bırak git
git gidebilirsen gönlümden
ben sabır taşı mıyım
döndürüp duruyorsun
değirmen taşı mıyım
yollarında ağladım
gündüzümü kolaymış
ne dileğim
senin gibi esmeliyim
dolaştırdım d

=======SEED========

temperature: 0.7107302122462241


sigara içen kişilerl

In [ ]:
def month_tr(month_):

    eng_months = 'February March April May June July August September October November December'.split()
    tr_months = 'Şubat Mart Nisan Mayıs Haziran Temmuz Ağustos Eylül Ekim Kasım Aralık Ocak'.split()

    month_index = eng_months.index(month_)
    tr_month = tr_months[month_index]
    
    return tr_month

In [ ]:
current_date = datetime.now().strftime('%d') + ' ' + month_tr(datetime.now().strftime('%B')) + ' ' + datetime.now().strftime('%Y')

for genre_ in lyrics_d:
    
    if genre_ == 'tsm':
        genre_readable = 'Türk sanat müziği temasında'

    elif genre_ == 'rock':
        genre_readable = 'Türkçe rock temasında'

    elif genre_ == 'pop':
        genre_readable = 'Türkçe pop temalı'

    elif genre_ == 'arabesk':
        genre_readable = 'Türkçe arabesk temalı'

    elif genre_ == 'rap':
        genre_readable = 'Türkçe rap temalı'

    # print(f'{current_date} tarihiyle, bunu takip eden {len(lyrics_d[genre_])} tweet, {genre_readable}')
    tweet_ = api.update_status(f'{current_date} tarihinde, bunu takip eden {len(lyrics_d[genre_])} tweet, {genre_readable}')
    time.sleep(1)

    for lyrics in lyrics_d[genre_]:

        last_space = lyrics.rfind(' ')
        last_pb = lyrics.rfind('\n')

        if last_pb < int(np.ceil(len(lyrics)/5)):
            # print(lyrics[:last_space])
            tweet_ = api.update_status(lyrics[:last_space])
        else:
            # print(lyrics[:last_pb])
            tweet_ = api.update_status(lyrics[:last_pb])
            time.sleep(1)